In [3]:
#install Beautiful Soup and requests for Web Scraping
!pip install BeautifulSoup4
!pip install requests

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
!pip install lxml

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


<h3> Step 1A: Get Data </h3>
<ol>
    <li>Get HTML from wikipedia</li>
    <li> Use BeautifySoup to parse html data </li>
    <li> Store parsed data into Pandas DataFrame </li>
</ol>

In [97]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [98]:
soup = BeautifulSoup(source.text, 'lxml')

In [99]:
print(source)

<Response [200]>


In [100]:
print(soup)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xc6xewpAME8AABFFwk0AAAAN","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":926306543,"wgRevisionId":926306543,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario

In [101]:
data = []
columns = []
table = soup.find(class_='wikitable')

In [102]:
print(table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [103]:
table.find_all('tr')

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_Yor

In [104]:
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if(index == 0):
        columns = section
    else:
        data.append(section)
        
#convert list into pandas dataframes
canada_df = pd.DataFrame(data = data, columns = columns )

In [105]:
canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [106]:
canada_df.shape

(287, 3)

<h3>Step 1B: Data Cleanup</h3>
<ol>
    <li> Remove Boroughs that are 'Not assigned' </li>
    <li> More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough</li>
</ol>

In [107]:
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']

In [108]:
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [109]:
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

In [110]:
canada_df = canada_df.drop_duplicates()
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Queen's Park,Not assigned


In [111]:
if(canada_df.index.name != 'Postcode'):
    canada_df = canada_df.set_index('Postcode')

In [112]:
canada_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [113]:
canada_df['Neighbourhood'].replace("Not assigned", canada_df['Borough'], inplace=True)

In [114]:
canada_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


<h3>Step 1C - Output</h3>
<p>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</p>

In [115]:
canada_df.shape

(103, 2)

<h3>Step 2</h3>


Downloading the dependencies that we will need

In [116]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [128]:
canada_df = canada_df.sort_values(by='Postcode')

In [130]:
canada_df.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [142]:
a = pd.read_csv('https://cocl.us/Geospatial_data')
gdf = pd.DataFrame(a)

In [143]:
gdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [145]:
if(gdf.index.name != 'Postal Code'):
    gdf = gdf.set_index('Postal Code')
    
if(canada_df.index.name != 'Postcode'):
    canada_df = canada_df.set_index('Postcode')

In [146]:
gdf[['Latitude','Longitude']].head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [147]:
result = pd.concat([canada_df, gdf[['Latitude','Longitude']]], axis=1, sort=False)

In [148]:
result

,Borough,Neighbourhood,Latitude,Longitude
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [152]:
result.index.name = 'PostalCode'

In [153]:
result

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [154]:
result.reset_index()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<h3>Part 3</h3>

In [228]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(result['Borough'].unique()),
        result.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [229]:
result.shape

(103, 4)

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.



In [164]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


Create a map of New York with neighborhoods superimposed on top.

In [166]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [168]:
central_toronto_data = result[result['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Davisville North,43.712751,-79.390197
2,Central Toronto,North Toronto West,43.715383,-79.405678
3,Central Toronto,Davisville,43.704324,-79.388790
4,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


Let's get the geographical coordinates of central toronto.

In [171]:
address = 'Central Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [172]:
CLIENT_ID = 'TKJKOR5BJCNI2XCNKPLSHNQYPTVL3L30OR3BYSJRYV4VGCMX' # your Foursquare ID
CLIENT_SECRET = 'DWC2NXL5ENBJYZVM3CNK0YRNC2CPAYHZI4HM3X0QD0E5OUW5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TKJKOR5BJCNI2XCNKPLSHNQYPTVL3L30OR3BYSJRYV4VGCMX
CLIENT_SECRET:DWC2NXL5ENBJYZVM3CNK0YRNC2CPAYHZI4HM3X0QD0E5OUW5


Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [173]:
central_toronto_data.loc[0, 'Neighbourhood']

'Lawrence Park'

Get the neighborhood's latitude and longitude values.

In [175]:
neighborhood_latitude = central_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [176]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=TKJKOR5BJCNI2XCNKPLSHNQYPTVL3L30OR3BYSJRYV4VGCMX&client_secret=DWC2NXL5ENBJYZVM3CNK0YRNC2CPAYHZI4HM3X0QD0E5OUW5&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

Send the GET request and examine the resutls

In [177]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd11a0a1e152c001b41b642'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [178]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.



In [179]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


And how many venues were returned by Foursquare?


In [180]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


<h3>Explore Neighborhoods in Central Toronto</h3>

Let's create a function to repeat the same process to all the neighborhoods in Manhattan¶


In [181]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called central_toronto_venues.¶


In [184]:
central_toronto_venues = getNearbyVenues(names=central_toronto_data['Neighbourhood'],
                                   latitudes=central_toronto_data['Latitude'],
                                   longitudes=central_toronto_data['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville


Let's check the size of the resulting dataframe

In [185]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(113, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


Let's check how many venues were returned for each neighborhood



In [186]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,34,34,34,34,34,34
Davisville North,9,9,9,9,9,9
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",15,15,15,15,15,15
"Forest Hill North, Forest Hill West",5,5,5,5,5,5
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",3,3,3,3,3,3
North Toronto West,20,20,20,20,20,20
Roselawn,2,2,2,2,2,2
"The Annex, North Midtown, Yorkville",22,22,22,22,22,22


Let's check how many venues were returned for each neighborhood



In [187]:
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 57 uniques categories.


<h3>Analyze Each Neighbourhood</h3>

In [189]:
# one hot encoding
central_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_toronto_onehot.columns[-1]] + list(central_toronto_onehot.columns[:-1])
central_toronto_onehot = central_toronto_onehot[fixed_columns]

central_toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Dessert Shop,Diner,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [190]:
central_toronto_onehot.shape

(113, 58)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶


In [191]:
central_toronto_grouped = central_toronto_onehot.groupby('Neighborhood').mean().reset_index()
central_toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Dessert Shop,Diner,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.058824,0.00,0.000000,0.058824,0.000000,0.000000,0.029412,0.088235,0.029412,0.029412,0.029412,0.000000,0.029412,0.0,0.00,0.029412,0.029412,0.00,0.058824,0.000000,0.0,0.000000,0.029412,0.058824,0.0,0.000000,0.000000,0.00,0.029412,0.029412,0.058824,0.000000,0.000000,0.00,0.029412,0.00,0.088235,0.029412,0.00,0.0,0.000000,0.000000,0.058824,0.000000,0.058824,0.029412,0.0,0.000000,0.000000,0.00
1,Davisville North,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.111111,0.000000,0.0,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.111111,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.00,0.111111,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.066667,0.066667,0.00,0.000000,0.000000,0.066667,0.000000,0.133333,0.00,0.066667,0.00,0.000000,0.000000,0.00,0.0,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.00
3,"Forest Hill North, Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.00
5,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.333333,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
6,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.050000,0.05,0.100000,0.100000,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.0,0.05,0.000000,0.000000,0.05,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0

Let's confirm the new size¶


In [192]:
central_toronto_grouped.shape

(9, 58)

<h3>Let's print each neighborhood along with the top 5 most common venues¶</h3>

In [193]:
num_top_venues = 5

for hood in central_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = central_toronto_grouped[central_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2                Café  0.06
3  Italian Restaurant  0.06
4    Sushi Restaurant  0.06


----Davisville North----
               venue  freq
0              Hotel  0.11
1                Gym  0.11
2  Convenience Store  0.11
3     Clothing Store  0.11
4     Sandwich Place  0.11


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07
3           Restaurant  0.07
4          Pizza Place  0.07


----Forest Hill North, Forest Hill West----
              venue  freq
0     Jewelry Store   0.2
1             Trail   0.2
2          Bus Line   0.2
3              Park   0.2
4  Sushi Restaurant   0.2


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1                 Park  0.33
2          Swim School  0.33
3  American Restaurant  0.00
4  

<h3>Let's put that into a pandas dataframe</h3>
First, let's write a function to sort the venues in descending order.

In [211]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.



In [216]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_toronto_grouped['Neighborhood']

for ind in np.arange(central_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Coffee Shop,Gym,Thai Restaurant,Sushi Restaurant,Pizza Place,Italian Restaurant,Café,Diner
1,Davisville North,Hotel,Clothing Store,Food & Drink Shop,Park,Pizza Place,Sandwich Place,Convenience Store,Gym,Breakfast Spot,Chinese Restaurant
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,American Restaurant,Supermarket,Liquor Store,Restaurant,Light Rail Station,Sports Bar,Pizza Place,Sushi Restaurant
3,"Forest Hill North, Forest Hill West",Trail,Jewelry Store,Sushi Restaurant,Bus Line,Park,Yoga Studio,Dessert Shop,Greek Restaurant,Gourmet Shop,Gift Shop
4,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Garden


<h3>Cluster Neighborhoods</h3>

Run k-means to cluster the neighborhood into 5 clusters.

In [217]:
# set number of clusters
kclusters = 5

central_toronto_grouped_clustering = central_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 4, 3, 0, 2, 1, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [218]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

central_toronto_merged = central_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
central_toronto_merged = central_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

central_toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Garden
1,Central Toronto,Davisville North,43.712751,-79.390197,2,Hotel,Clothing Store,Food & Drink Shop,Park,Pizza Place,Sandwich Place,Convenience Store,Gym,Breakfast Spot,Chinese Restaurant
2,Central Toronto,North Toronto West,43.715383,-79.405678,2,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gift Shop,Diner,Mexican Restaurant,Park,Dessert Shop,Grocery Store
3,Central Toronto,Davisville,43.704324,-79.388790,2,Dessert Shop,Sandwich Place,Coffee Shop,Gym,Thai Restaurant,Sushi Restaurant,Pizza Place,Italian Restaurant,Café,Diner
4,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Gym,Restaurant,Playground,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Garden,Fried Chicken Joint,Food & Drink Shop


Finally let's visualise resulting clusters

In [220]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_toronto_merged['Latitude'], central_toronto_merged['Longitude'], central_toronto_merged['Neighbourhood'], central_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters</h3>
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

Cluster 1

In [222]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 0, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Moore Park, Summerhill East",Gym,Restaurant,Playground,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Garden,Fried Chicken Joint,Food & Drink Shop


Cluster 2

In [223]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 1, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Roselawn,Home Service,Garden,Yoga Studio,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Fried Chicken Joint,Food & Drink Shop


Cluster 3

In [224]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 2, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Davisville North,Hotel,Clothing Store,Food & Drink Shop,Park,Pizza Place,Sandwich Place,Convenience Store,Gym,Breakfast Spot,Chinese Restaurant
2,North Toronto West,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gift Shop,Diner,Mexican Restaurant,Park,Dessert Shop,Grocery Store
3,Davisville,Dessert Shop,Sandwich Place,Coffee Shop,Gym,Thai Restaurant,Sushi Restaurant,Pizza Place,Italian Restaurant,Café,Diner
5,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,American Restaurant,Supermarket,Liquor Store,Restaurant,Light Rail Station,Sports Bar,Pizza Place,Sushi Restaurant
8,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Coffee Shop,American Restaurant,Pharmacy,Pizza Place,Liquor Store,Pub,Cosmetics Shop,Flower Shop


Cluster 4

In [225]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 3, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Garden


Cluster 5

In [227]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 4, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Forest Hill North, Forest Hill West",Trail,Jewelry Store,Sushi Restaurant,Bus Line,Park,Yoga Studio,Dessert Shop,Greek Restaurant,Gourmet Shop,Gift Shop
